In [ ]:
"""
This file is part of mr-recv-coil-match-networks.
Copyright © 2024 Technical University of Denmark (developed by Rasmus Jepsen)

This library is free software; you can redistribute it and/or
modify it under the terms of the GNU Lesser General Public
License as published by the Free Software Foundation; either
version 2.1 of the License, or (at your option) any later version.

This library is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public
License along with this library; if not, write to the Free Software
Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301  USA
"""

In [ ]:
"""
This notebook shows that an upside-down extended bridged T 9 network topology network cannot simultaneously provide noise matching, preamplifier decoupling and common mode rejection.

This notebook uses a similar process to the derivations in [1].
"""

In [ ]:
# import modules
from lcapy import Circuit, oo, expr, symbol, j, Eq, Z, Matrix
from lcapy.expr import symbols

In [ ]:
# construct and draw network
matching_net = Circuit("""
W 1 5_0; right
Z3 5_0 4_0; down
Z1 3_0 4_0; right
Z4 4_0 0; right
W 5_0 2; right
W 3_0 3_1; down
W 0 0_1; down
W 3_1 3_2; right
Z2 3_2 0_1; right
Z5 3 3_0; right
""")
matching_net.draw()

In [ ]:
# differential-mode two-port form of the network
matching_net_twoport = matching_net.twoport(1, 3, 2, 0)

In [ ]:
# initialise symbols

# resistance and reactance of the optimal output impedance for noise matching
r_out, x_out = symbols('R_out X_out', real=True)

# resistance and reactance of the coil
r_coil, x_coil = symbols('R_coil X_coil', real=True)

# input resistance and reactance of the preamplifier
r_amp, x_amp = symbols('R_amp X_amp', real=True)

# input impedance of the preamplifier and the coil impedance
z_amp, z_coil = symbols('Z_amp Z_coil', complex=True)

# element reactances
x_1, x_2, x_3, x_4, x_5 = symbols('X_1 X_2 X_3 X_4 X_5', real=True)

In [ ]:
# three-port Z parameters
threeport_z = matching_net.Zparamsn(2,0,1,0,3,0)
threeport_z

In [ ]:
# The technique described in [2] is used to convert the three-port impedance matrix to three-port scattering parameters.

In [ ]:
# initialise matrices for converting Z-parameters to S-parameters

f = Matrix(((z_amp.real ** 0.5 / 2, 0, 0), (0, (z_coil.real / 2) ** 0.5 / 2, 0), (0, 0, (z_coil.real / 2) ** 0.5 / 2)))

g = Matrix(((z_amp, 0, 0), (0, z_coil / 2, 0), (0, 0, z_coil / 2)))

g_plus = Matrix(((z_amp.conj, 0, 0), (0, z_coil.conj / 2, 0), (0, 0, z_coil.conj / 2)))

In [ ]:
# calculate the renormalised three-port scattering parameters
threeport_s = f * (threeport_z - g_plus) * (threeport_z + g).inv() * f.inv()
threeport_s

In [ ]:
# The method described in [3] is used to calculate the common-mode rejection ratio (CMRR).

In [ ]:
# common-mode response
matching_net_s21cs = expr('1/sqrt(2)') * (threeport_s[1,0] + threeport_s[2,0])
matching_net_s21cs.simplify()

In [ ]:
# differential-mode response
matching_net_s21ds = expr('1/sqrt(2)') * (threeport_s[1,0] - threeport_s[2,0])
matching_net_s21ds.simplify()

In [ ]:
# variable substitutions for later steps
substitutions = {'Z1': j * x_1, 'Z2': j * x_2, 'Z3': j * x_3, 'Z4': j * x_4, 'Z5': j * x_5, 'Z_coil': r_coil + j * x_coil, 'Z_amp': r_amp + j * x_amp}

In [ ]:
# calculate the inverse of the common-mode rejection ratio and substitute values in
inv_cmrr = (matching_net_s21cs / matching_net_s21ds).simplify().subs(substitutions).simplify()
inv_cmrr

In [ ]:
# solve element reactances for when the inverse CMRR is 0, which corresponds to an infinite CMRR
# numerator of inv_cmrr is used to simplify the task for the symbolic solver
cmrr_system = expr([Eq(inv_cmrr.N, 0)])
cmrr_unknowns = expr((x_1, x_2, x_3, x_4, x_5))
cmrr_solutions = cmrr_system.solve(cmrr_unknowns)
cmrr_solutions

In [ ]:
# check that the solution yields an inverse CMRR of 0
inv_cmrr_subbed = list(inv_cmrr.subs(list((unknown, sol[i]) for i, unknown in enumerate(cmrr_unknowns))).simplify() for sol in cmrr_solutions)
inv_cmrr_subbed

In [ ]:
# define symbols for the two-port reactance parameters
x_11, x_12, x_22 = symbols('X_11 X_12 X_22', real=True)

In [ ]:
# differential-mode two-port impedance parameters for the network given common-mode rejection
twoport_z = matching_net_twoport.Zparams.subs(substitutions).subs(x_1, cmrr_solutions[0][0]).subs(x_2, cmrr_solutions[0][1]).simplify()
twoport_z

In [ ]:
twoport_z11 = twoport_z[0, 0]
twoport_z11

In [ ]:
twoport_z12 = twoport_z[0, 1]
twoport_z12

In [ ]:
twoport_z22 = twoport_z[1, 1]
twoport_z22

In [ ]:
# The only solution that provides common-mode rejection yields Z_11 = 2*Z_12 = 4*Z_22.
# Therefore, this topology cannot match arbitrary lossless reciprocal impedance parameters from the equations described in [4] and
# cannot be used as a receive coil matching network to simultaneously provide noise matching, optimal preamplifier decoupling and common-mode rejection.

In [ ]:
"""References:
[1] R. A. Jepsen, "LC Power-Matching Baluns". Zenodo, Feb. 28, 2024. doi: 10.5281/zenodo.10723786.
[2] K. Kurokawa, "Power waves and the scattering matrix," IEEE Transactions on Microwave Theory and Techniques, vol. 13, no. 2, pp. 194–202, 1965.
[3] D. Bockelman and W. Eisenstadt, "Combined differential and common-mode analysis of power splitters and combiners," IEEE Transactions on Microwave Theory and Techniques, vol. 43, no. 11, pp. 2627–2632, 1995.
[4] W. Wang, V. Zhurbenko, J. D. Sánchez‐Heredia, and J. H. Ardenkjær‐Larsen, "Trade‐off between preamplifier noise figure and decoupling in MRI detectors," Magnetic Resonance in Medicine, vol. 89, no. 2, pp. 859–871, 2023. doi:10.1002/mrm.29489
"""